# Dynamic Programming Corner Growth

In [1]:
# Some of the code in this notebook is taken from "RandomGrowth" in the notebooks folder of this repository.

In [2]:
using Random, Distributions, Plots, Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-9419618213350070821\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16499807952727872579\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-9419618213350070821\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
function get_corner_growth_times(independent_draws)
    # Simulate corner growth with Dynamic Programming
    # params:
    # independent_draws: a matrix of independent draws from any distribution
    # T(m, n) = max(T(m-1, n), T(m, n-1)) + s + 1
    (num_rows, num_cols) = size(independent_draws)
    T = zeros(num_rows, num_cols)
    for row_num = 1:num_rows
        for col_num = 1:num_cols
            if (row_num > 1 && col_num > 1)
                # This is really the main case; the rest are just literally the "edges."
                T[row_num, col_num] = (max(T[row_num - 1, col_num], T[row_num, col_num - 1])
                    + independent_draws[row_num, col_num] + 1)
            elseif (row_num == 1 && col_num > 1)
                T[row_num, col_num] = (T[row_num, col_num - 1] + independent_draws[row_num, col_num]
                    + 1)
            elseif (col_num == 1 && row_num > 1)
                T[row_num, col_num] = (T[row_num - 1, col_num] + independent_draws[row_num, col_num]
                    + 1)
            elseif (col_num == 1 && row_num == 1)
                T[row_num, col_num] = independent_draws[row_num, col_num] + 1
            else
                error("Invalid row or column number combination // out of bounds.")
            end
        end
    end
    T = T .- (T[1,1] - 1) # so that T[1,1] = 1
    return T
end

get_corner_growth_times (generic function with 1 method)

In [4]:
# Visualize corner growth

In [68]:
p = .6
q = 1 - p
num_rows = 30
num_cols = 30
independent_draws = rand(Geometric(p), num_rows, num_cols)
T = get_corner_growth_times(independent_draws)
# analytical limiting shape:
limiting(x, y) = x + y + 2*sqrt(x*y*q);

In [69]:
@manipulate for time_step in slider(1:maximum(T)+1, value=1)
    # plot which squares are "alive"
    heatmap((T .< time_step) + (T .== time_step) .* 0.5, # This makes the new squares a different color
        colorbar = false, aspect_ratio = 1, clims = (0,1),
        xlims = (1, num_cols), ylims = (1, num_rows))
    # plot the theoretical contour
    scaling = time_step + 2; # Is this scaling correct? Taken from the RandomGrowth notebook in this repo 
    xs = scaling .* (0:0.01:(1-q));
    ys = scaling .* (0:0.01:(1-q));
    contour!(xs, ys, (x,y)->limiting(x/scaling,y/scaling), colorbar=false, levels = [1-q],
            color=:blue, lw=3)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 151,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/mitchell/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000012dc86650, Task (runnable) @0x000000012dc86650), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1.0\",\"2.0\",\"3.0\",\"4.0\",\"5.0\",\"6.0\",\"7.0\",\"8.0\",\"9.0\",\"10.0\",\"11.0\",\"12.0\",\"13.0\",\"14.0\",\"15.0\",\"16.0\",\"17.0\",\"18.0\",\"19.0\",\"20.0\",\"21.0\",\"22.0\",\"23.0\",\"24.0\",\"25.0\",\"26.0\",\"27.0

In [70]:
# Visualize on a scaled grid
max_time = min(maximum(T[:,1]), maximum(T[1,:]))
# You can choose any max time step. This is the biggest that is fully "extended" along axes.
@manipulate for time_step in slider(2:max_time, value=2)
    time_scaled_grid = ones(Int(time_step), Int(time_step)) * Inf # want this to be larger than any time step
    # To get the scaled grid we have to embed everything this in a grid that scales with time step count.
    biggest_that_fits = sum((T .< time_step)[:,1])
    time_scaled_grid[1:biggest_that_fits, 1:biggest_that_fits] = T[1:biggest_that_fits, 1:biggest_that_fits]
    p = heatmap(0:(1/(time_step-1)):1, 0:(1/(time_step-1)):1,
        (time_scaled_grid .< time_step) + (time_scaled_grid .== time_step) * 0.5,
        aspect_ratio=1, colorbar=false, clims=(0, 1), xlims = (0, 1), ylims=(0, 1))
    range = 0:0.001:1
    contour!(range, range,
            (x, y) -> limiting(x, y), levels=[1-q], colorbar=false, 
             color=:blue, lw=3)
    p
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 59,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/mitchell/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/mitchell/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/mitchell/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x000000012dc88490, Task (runnable) @0x000000012dc88490), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2.0\",\"3.0\",\"4.0\",\"5.0\",\"6.0\",\"7.0\",\"8.0\",\"9.0\",\"10.0\",\"11.0\",\"12.0\",\"13.0\",\"14.0\",\"15.0\",\"16.0\",\"17.0\",\"18.0\",\"19.0\",\"20.0\",\"21.0\",\"22.0\",\"23.0\",\"24.0\",\"25.0\",\"26.0\",\"27.0\",\"28.0